## overview

In [1]:
# default package
import logging
import sys 
import os
import pathlib
import IPython
import random

In [2]:
# third party package
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import psycopg2
import sqlalchemy
from sqlalchemy.sql.expression import column,select,table
from sqlalchemy import create_engine
import sklearn.datasets

In [3]:
# my package
sys.path.append(os.path.join(pathlib.Path().resolve(),"../"))

In [4]:
# reload settings
%load_ext autoreload
%autoreload 2

In [5]:
# logger
logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [6]:
## grap setting
sns.set()

In [7]:
## db setting
engine = create_engine('postgresql://postgres:postgres@db:5432/postgres')
with engine.connect() as conn:
    print(conn.closed)

False


## sql to python

In [8]:
def ct(file:str,chap_num:int,num:int,sub_num:int=None,engine:sqlalchemy.engine=engine)->None:
    """
    pubulic schema内のtableをすべて削除してから、指定のtable作成
    """
    if sub_num==None: 
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}"
    else:
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}_{sub_num}"
        
    with open(filename,"r") as f:
        stmt=f.read() 
    with engine.connect() as conn:
        conn.execute("drop schema public cascade; create schema public;")
        conn.execute(stmt)

In [9]:
def exe(stmt:str,engine:sqlalchemy.engine=engine):
    """
    sql実行
    """
    with engine.connect() as conn:
        query=conn.execute(stmt)
        try:
            result=query.fetchall()
            return result
        except:
            return None

## List

In [10]:
ct("List",3,1)
exe("""delete from 家計簿 where 出金額>0""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0)]

In [11]:
ct("List",3,3)
exe("""select * from 家計簿 where メモ like '%%1月%%'""")

[(datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560)]

In [12]:
ct("List",3,4)
exe("""
select * from 家計簿 where 出金額 between 100 and 3000
""")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800)]

In [13]:
ct("List",3,4)
exe("""select * from 家計簿 where 出金額>3000""")

[(datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560)]

In [14]:
ct("List",3,4)
exe("""select * from 家計簿 where 費目 in ('食費','交際費')""")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000)]

In [15]:
ct("List",3,4)
exe("""select * from 家計簿 where 費目 not in ('食費','交際費')""")

[(datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560)]

In [16]:
ct("List",3,7)
exe("""
update 湊くんの買い物リスト
set 価格=6200
where 販売店='B' and 名称='スッキリ勇者クエスト'
""")
exe("select * from 湊くんの買い物リスト")

[('ゲーム', 'スッキリ勇者クエスト', 'Y', 6850),
 ('書籍', '魔王征伐日記', 'A', 1200),
 ('DVD', 'スッキリわかるマンモスの倒し方', 'A', 5250),
 ('DVD', 'スッキリわかるマンモスの倒し方', 'B', 7140),
 ('ゲーム', 'スッキリ勇者クエスト', 'B', 6200)]

In [17]:
ct("List",3,9)
exe("""
select * from 湊くんの買い物リスト
where (販売店='A' or 販売店='B')
and 
(カテゴリ='ゲーム' or カテゴリ='DVD')
""")

[('ゲーム', 'スッキリ勇者クエスト', 'B', 7140),
 ('DVD', 'スッキリわかるマンモスの倒し方', 'A', 5250),
 ('DVD', 'スッキリわかるマンモスの倒し方', 'B', 7140)]

In [18]:
ct("List",3,10)
exe("""
delete from 社員
where 社員番号='2003031'
""")
exe("select * from 社員")

[('2003032', 45, '1', 'ヨシダ\u3000シゲル'),
 ('2005011', 31, '1', 'スガワラ\u3000タクマ'),
 ('2017001', 22, '1', 'ミナト\u3000ユウスケ'),
 ('2017002', 24, '2', 'アサカ\u3000アユミ')]

## Q

In [19]:
ct("Q",3,1,1)
exe("""select * from 気象観測
where 月=6
""")

[(6, 236, 32, 16, 73)]

In [20]:
ct("Q",3,1,2)
exe("""select * from 気象観測
where 月<>6
""")

[(1, 31, 11, -1, 43),
 (2, 115, 13, -3, 49),
 (3, 142, 20, -1, 59),
 (4, 118, 30, 1, 63),
 (5, 44, 30, 6, 65),
 (7, 333, 37, 18, 75),
 (8, 112, 37, 21, 69),
 (9, 140, 34, 17, 73),
 (10, 88, 28, 8, 65),
 (11, 120, 19, 2, 58),
 (12, 44, 14, -2, 50)]

In [21]:
ct("Q",3,1,1)
exe("""select * from 気象観測
where 降水量<100
""")

[(1, 31, 11, -1, 43),
 (5, 44, 30, 6, 65),
 (10, 88, 28, 8, 65),
 (12, 44, 14, -2, 50)]

In [22]:
ct("Q",3,1,4)
exe("""select * from 気象観測
where 降水量>200
""")

[(6, 236, 32, 16, 73), (7, 333, 37, 18, 75)]

In [23]:
ct("Q",3,1,"7_1")
exe("""select * from 気象観測
where 月 in (3,5,7)
""")

[(3, 142, 20, -1, 59), (5, 44, 30, 6, 65), (7, 333, 37, 18, 75)]

In [24]:
ct("Q",3,1,"8_1")
exe("""select * from 気象観測
where 月 not in (3,5,7)
""")

[(1, 31, 11, -1, 43),
 (2, 115, 13, -3, 49),
 (4, 118, 30, 1, 63),
 (6, 236, 32, 16, 73),
 (8, 100, 37, 21, 69),
 (9, 140, 34, 17, 73),
 (10, 88, 28, 8, 65),
 (11, 120, 19, 2, 58),
 (12, 44, 14, -2, 50)]

In [25]:
ct("Q",3,1,"11_1")
exe("""select * from 気象観測
where 湿度 between 60 and 79
""")

[(4, 118, 30, 1, 63),
 (5, 44, 30, 6, 65),
 (6, 236, 32, 16, 73),
 (7, 333, 37, 18, 75),
 (8, 100, 37, 21, 69),
 (9, 140, 34, 17, 73),
 (10, 88, 28, 8, 65)]

In [26]:
ct("Q",3,2,1)
exe("""select * from 都道府県
where 都道府県名 like '%%川'
""")

[('14', '関東', '神奈川', '横浜', 2415), ('37', '四国', '香川', '高松', 1876)]

In [27]:
ct("Q",3,2,1)
exe("""select * from 都道府県
where 都道府県名 like '%%島%%'
""")

[('34', '中国', '広島', '広島', 8478)]

In [28]:
ct("Q",3,2,1)
exe("""select * from 都道府県
where 都道府県名=県庁所在地
""")

[('05', '東北', '秋田', '秋田', 11612),
 ('16', '北陸', '富山', '富山', 4247),
 ('20', '甲信', '長野', '長野', 13562),
 ('34', '中国', '広島', '広島', 8478),
 ('26', '近畿', '京都', '京都', 4613),
 ('40', '九州', '福岡', '福岡', 4976)]

In [29]:
ct("Q",3,2,1)
exe("""select * from 都道府県
where 都道府県名<>県庁所在地
""")

[('01', '北海道', '北海道', '札幌', 83456),
 ('14', '関東', '神奈川', '横浜', 2415),
 ('23', '東海', '愛知', '名古屋', 5165),
 ('37', '四国', '香川', '高松', 1876)]

In [30]:
ct("Q",3,3,1)
exe("""select * from 成績表""")

[('S002', '織田\u3000信長', 50, 60, 70, 80, 90, None),
 ('A001', '豊臣\u3000秀吉', 45, 55, 65, 75, 85, None),
 ('E004', '徳川\u3000家康', 90, 80, 70, 60, 50, None)]

In [31]:
ct("Q",3,3,2)
exe("""
insert into 成績表 values('S001','織田信長',77,55,80,75,93,NULL)
""")
exe("""select * from 成績表""")

[('S002', '織田 信長', 50, 60, 70, 80, 90, None),
 ('A001', '豊臣 秀吉', 45, 55, 65, 75, 85, None),
 ('E004', '徳川 家康', 90, 80, 70, 60, 50, None),
 ('S001', '織田信長', 77, 55, 80, 75, 93, None)]

In [32]:
ct("Q",3,3,3)
exe("""
update 成績表 set 法学=85,哲学=67
where 学籍番号='S001'
""")
exe("""select * from 成績表""")

[('S002', '織田\u3000信長', 50, 60, 70, 80, 90, None),
 ('A001', '豊臣\u3000秀吉', 45, 55, 65, 75, 85, None),
 ('E004', '徳川\u3000家康', 90, 80, 70, 60, 50, None),
 ('A002', '豊臣\u3000秀吉', 64, 69, 70, 0, 59, None),
 ('E003', '徳川\u3000家康', 80, 83, 85, 90, 79, None),
 ('S001', '織田\u3000信長', 85, 55, 67, 75, 93, None)]

In [33]:
ct("Q",3,3,3)
exe("""
update 成績表 set 外国語=81
where 学籍番号 in ('A002','E003')
""")
exe("""select * from 成績表""")

[('S002', '織田\u3000信長', 50, 60, 70, 80, 90, None),
 ('A001', '豊臣\u3000秀吉', 45, 55, 65, 75, 85, None),
 ('E004', '徳川\u3000家康', 90, 80, 70, 60, 50, None),
 ('S001', '織田\u3000信長', 77, 55, 80, 75, 93, None),
 ('A002', '豊臣\u3000秀吉', 64, 69, 70, 0, 81, None),
 ('E003', '徳川\u3000家康', 80, 83, 85, 90, 81, None)]